In [1]:
# Import dependencies 
import numpy as np
import pandas as pd
import csv

import sqlite3

from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine

# Machine learning libraries (included inline in code)
#from sklearn.model_selection import train_test_split 
#from sklearn.ensemble import RandomForestRegressor
#from sklearn.metrics import mean_squared_error, r2_score
#from sklearn.model_selection import RandomizedSearchCV
#from sklearn.model_selection import GridSearchCV

# Pickle will allow us to save our model in a usable manner 
import pickle 

# Setting engine for SQLite connection
engine = create_engine("sqlite:///birthdata.sqlite", echo=False)


## Starting SQLite Connection

In [2]:
# Checking classes were made successfully: https://stackoverflow.com/questions/42946174/sqlalchemy-automap-not-generating-base-classes-table-name
engine = create_engine("sqlite:///birthdata.sqlite", echo=False)

# Declare a Base using `automap_base()`
Base = automap_base()

# Use the Base class to reflect the database tables
Base.prepare(engine, reflect=True)

# Print all of the classes mapped to the Base
print(Base.classes.keys())


['clinic2010_county', 'clinic2010_state', 'clinic2015_county', 'clinic2015_state', 'county', 'county_pop', 'county_svi', 'national', 'outcomes']


### Naming Classes

In [3]:
# Create a session
session = Session(engine)

# Assign the classes to variables
# New classes 
County2015 = Base.classes.clinic2015_county
State2015 = Base.classes.clinic2015_state
County2010 = Base.classes.clinic2010_county
State2010 = Base.classes.clinic2010_state
Outcomes = Base.classes.outcomes
Countypop = Base.classes.county_pop
CountySVI = Base.classes.county_svi

# Prior classes 
County_births = Base.classes.county
State_births = Base.classes.national


## Querying SQLite

In [4]:
# Using a join in our query to pull several tables together
# Resource: https://www.kite.com/python/answers/how-to-join-multiple-tables-together-in-sqlalchemy-in-python#:~:text=Use%20Query.,sequence%20to%20tables%20to%20join.
join_query = session.query(County2015.fips, County2015.total_clinics, County2015.total_titleten, County2015.pp,\
                           County2015.dept_clinic, County2015.hospital, County2015.total_client_tt,\
                           County2015.pp_client, County2015.dept_clinic_tt, County2015.pp_tt,\
                           County2015.total_client, County2015.hospital_client, County_births.birth_rate,\
                           County_births.year, County_births.state, County_births.county)\
                    .join(County_births, County_births.combined_fips_code == County2015.fips)\
                    .filter(County_births.year=="2016")


county_df = pd.DataFrame(join_query, columns=["FIPS", "total_clinics", "total_title10", "total_pp", "health_dept_clinics", 
                                              "hospitals","title_10_clients","pp_clients", "dept_clinic_title10","pp_tt",
                                              "total_clients","hospital_client","birth_rate", "year", "state", "county"])

# Drop NaN rows, which will mess with the ML
county_df = county_df.dropna()


In [5]:
session.close()

county_df.head()

,FIPS,total_clinics,total_title10,total_pp,health_dept_clinics,hospitals,title_10_clients,pp_clients,dept_clinic_title10,pp_tt,total_clients,hospital_client,birth_rate,year,state,county
0,1001,2,1,0,1,0,870.0,0,1,0,1040.0,0,23.1,2016,Alabama,Autauga
1,1003,6,2,0,2,0,990.0,0,2,0,2010.0,0,25.6,2016,Alabama,Baldwin
2,1005,3,2,0,2,0,900.0,0,2,0,940.0,0,36.6,2016,Alabama,Barbour
3,1007,5,1,0,1,0,510.0,0,1,0,710.0,0,36.5,2016,Alabama,Bibb
4,1009,2,1,0,1,0,1200.0,0,1,0,1290.0,0,30.6,2016,Alabama,Blount


In [6]:
county_df.shape

(3100, 16)

In [7]:
county_df["year"].unique()

array([2016])

## Adding in Population Data by County 

In [8]:
# Use Pandas read sql to grab the entire table into a df 
county_populations = pd.read_sql_query('SELECT * FROM county_pop', con=engine)
county_populations.set_index('index', inplace=True)
county_populations


,state,county,population_2010,population_2015
index,,,,
0,Iowa,Adair,7682,7145
1,Kentucky,Adair,18656,19162
2,Missouri,Adair,25607,25353
3,Oklahoma,Adair,22683,22259
4,Colorado,Adams,441603,490443
...,...,...,...,...
3136,California,Yuba,72155,74045
3137,Alaska,Yukon-Koyukuk,5588,5465
3138,Texas,Zapata,14018,14493


In [9]:
# Merging this population data with the prior df 
county_df = county_df.merge(county_populations, how='left', on=["state","county"])


## Adding in Social Vulnerability Index Data 

In [10]:
# Use Pandas read sql to grab the entire table into a df 
county_SVI = pd.read_sql_query('SELECT * FROM county_svi', con=engine)
county_SVI.set_index('index', inplace=True)
county_SVI=county_SVI.drop(columns=["state","county"])
county_SVI


,FIPS,SVI_sum_of_indicators,SVI_ranking,percent_uninsured
index,,,,
0,8093,1.9147,0.0000,11.6
1,17133,2.4187,0.0003,4.2
2,31115,2.5091,0.0006,9.2
3,32011,2.5842,0.0010,16.6
4,30069,2.5985,0.0013,22.9
...,...,...,...,...
3137,48507,12.2951,0.9987,21.7
3138,48047,12.3448,0.9990,23.5
3139,35006,12.3728,0.9994,19.0


In [11]:
# Merging this SVI data with the first df 
county_df=county_df.merge(county_SVI, how='left', on="FIPS")


## Per Capita Calculations

In [12]:
# Transforming data to a per capita basis
county_df["clinics_per_capita"] = county_df["total_clinics"]/county_df["population_2015"]
county_df["title10_clinics_per_capita"] = county_df["total_title10"]/county_df["population_2015"]
county_df["pp_per_capita"] = county_df["total_pp"]/county_df["population_2015"]
county_df["health_dept_per_capita"] = county_df['health_dept_clinics']/county_df["population_2015"]
county_df["hospitals_per_capita"] = county_df['hospitals']/county_df["population_2015"]
county_df["title_10_clients_per_capita"] = county_df["title_10_clients"]/county_df["population_2015"]
county_df["pp_clients_per_capita"] = county_df['pp_clients']/county_df["population_2015"]
county_df["dept_clinic_title10_per_capita"] = county_df['dept_clinic_title10']/county_df["population_2015"]


In [13]:
county_df = county_df.dropna()
county_df.shape

(3071, 29)

In [14]:
county_df.head()

,FIPS,total_clinics,total_title10,total_pp,health_dept_clinics,hospitals,title_10_clients,pp_clients,dept_clinic_title10,pp_tt,...,SVI_ranking,percent_uninsured,clinics_per_capita,title10_clinics_per_capita,pp_per_capita,health_dept_per_capita,hospitals_per_capita,title_10_clients_per_capita,pp_clients_per_capita,dept_clinic_title10_per_capita
0,1001,2,1,0,1,0,870.0,0,1,0,...,0.3773,8.9,0.000036,0.000018,0.0,0.000018,0.0,0.015857,0.0,0.000018
1,1003,6,2,0,2,0,990.0,0,2,0,...,0.2757,11.8,0.000030,0.000010,0.0,0.000010,0.0,0.004878,0.0,0.000010
2,1005,3,2,0,2,0,900.0,0,2,0,...,0.9847,13.0,0.000114,0.000076,0.0,0.000076,0.0,0.034243,0.0,0.000076
3,1007,5,1,0,1,0,510.0,0,1,0,...,0.5737,9.0,0.000222,0.000044,0.0,0.000044,0.0,0.022600,0.0,0.000044
4,1009,2,1,0,1,0,1200.0,0,1,0,...,0.4986,11.2,0.000035,0.000017,0.0,0.000017,0.0,0.020860,0.0,0.000017


## Crafting the Machine Learning Model 

In [34]:
county_fips = "17031"

In [35]:
calculator_prefill = county_df.loc[county_df["FIPS"] == int(county_fips), ['FIPS','birth_rate','clinics_per_capita','title10_clinics_per_capita', 'pp_per_capita', 
        'health_dept_per_capita', 'hospitals_per_capita', 'title_10_clients_per_capita','pp_clients_per_capita', 
        'dept_clinic_title10_per_capita','percent_uninsured','SVI_sum_of_indicators']]
calculator_prefill


,FIPS,birth_rate,clinics_per_capita,title10_clinics_per_capita,pp_per_capita,health_dept_per_capita,hospitals_per_capita,title_10_clients_per_capita,pp_clients_per_capita,dept_clinic_title10_per_capita,percent_uninsured,SVI_sum_of_indicators
595,17031,21.4,0.000033,0.000007,0.000002,5.720728e-07,0.000004,0.013669,0.006232,1.906909e-07,12.6,8.6167


In [36]:
prefill_values=calculator_prefill.values 
values=prefill_values[0]

In [38]:
values

array([1.70310000e+04, 2.14000000e+01, 3.33709122e-05, 6.86487336e-06,
       1.52552741e-06, 5.72072780e-07, 3.81381853e-06, 1.36687256e-02,
       6.23177948e-03, 1.90690927e-07, 1.26000000e+01, 8.61670000e+00])

In [15]:
# Prepping the model
# X is our data. y is our target. 
X = county_df[['clinics_per_capita',
       'title10_clinics_per_capita', 'pp_per_capita', 'health_dept_per_capita',
       'hospitals_per_capita', 'title_10_clients_per_capita',
       'pp_clients_per_capita', 'dept_clinic_title10_per_capita','percent_uninsured','SVI_sum_of_indicators']]
y = county_df["birth_rate"] 

feature_names = ['clinics_per_capita',
       'title10_clinics_per_capita', 'pp_per_capita', 'health_dept_per_capita',
       'hospitals_per_capita', 'title_10_clients_per_capita',
       'pp_clients_per_capita', 'dept_clinic_title10_per_capita','percent_uninsured','SVI_sum_of_indicators']

print("Shape: ", X.shape, y.shape)


Shape:  (3071, 10) (3071,)


## Train Test Split

In [16]:
# Divide our data for proper training/testing subsections
from sklearn.model_selection import train_test_split 

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)


### Base Model 

In [17]:
# Create base model
from sklearn.ensemble import RandomForestRegressor

base_model = RandomForestRegressor(max_depth=7, n_estimators=100, random_state=1)
base_model.fit(X_train, y_train)


RandomForestRegressor(max_depth=7, random_state=1)

### Mean Squared Error, R2 Score, and Feature Importances

In [18]:
from sklearn.metrics import mean_squared_error, r2_score

# Use our model to make predictions
base_model_predicted = base_model.predict(X_test)

# Score the predictions with mse and r2
mse = mean_squared_error(y_test, base_model_predicted)
r2 = r2_score(y_test, base_model_predicted)

print(f"Mean Squared Error (MSE): {mse}")
print(f"R-squared (R2 ): {r2}")

Mean Squared Error (MSE): 81.95427809246755
R-squared (R2 ): 0.5053740044409091


In [19]:
# Use "feature importances" to see which data columns are weightier 
sorted(zip(base_model.feature_importances_, feature_names), reverse=True)


[(0.7023997185548906, 'SVI_sum_of_indicators'),
 (0.09968606609982379, 'percent_uninsured'),
 (0.04802607734822058, 'pp_clients_per_capita'),
 (0.033695275236732776, 'pp_per_capita'),
 (0.03125840246242696, 'clinics_per_capita'),
 (0.021647816730657146, 'health_dept_per_capita'),
 (0.021457949270325172, 'title10_clinics_per_capita'),
 (0.0209969622034526, 'title_10_clients_per_capita'),
 (0.017878148764163024, 'dept_clinic_title10_per_capita'),
 (0.002953583329307376, 'hospitals_per_capita')]

### Randomized Search 

In [20]:
# Resource for gridsearch with random forest: https://towardsdatascience.com/hyperparameter-tuning-the-random-forest-in-python-using-scikit-learn-28d2aa77dd74a77dd74
# Start with a randomized search to narrow down the field
from sklearn.model_selection import RandomizedSearchCV

params = {'bootstrap': [True, False],
         'max_depth': [3,5,10,20,50, None],
         'max_features': ['auto', 'sqrt'],
         'min_samples_leaf': [1, 2, 4],
         'min_samples_split': [2, 5, 10],
         'n_estimators': [100,200, 400, 600, 800, 1000, 1250, 1500, 2000]}

rf_random = RandomizedSearchCV(estimator = RandomForestRegressor(), param_distributions = params, n_iter = 100, cv = 3, verbose=2, random_state=42, n_jobs = -1)
rf_random.fit(X_train, y_train)


Fitting 3 folds for each of 100 candidates, totalling 300 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  33 tasks      | elapsed:   59.6s
[Parallel(n_jobs=-1)]: Done 154 tasks      | elapsed:  3.9min
[Parallel(n_jobs=-1)]: Done 300 out of 300 | elapsed:  7.5min finished


RandomizedSearchCV(cv=3, estimator=RandomForestRegressor(), n_iter=100,
                   n_jobs=-1,
                   param_distributions={'bootstrap': [True, False],
                                        'max_depth': [3, 5, 10, 20, 50, None],
                                        'max_features': ['auto', 'sqrt'],
                                        'min_samples_leaf': [1, 2, 4],
                                        'min_samples_split': [2, 5, 10],
                                        'n_estimators': [100, 200, 400, 600,
                                                         800, 1000, 1250, 1500,
                                                         2000]},
                   random_state=42, verbose=2)

In [21]:
rf_random.best_params_

{'n_estimators': 2000,
 'min_samples_split': 10,
 'min_samples_leaf': 4,
 'max_features': 'sqrt',
 'max_depth': 10,
 'bootstrap': True}

In [22]:
random_tuned_model = rf_random.best_estimator_

In [23]:
random_tuned_predictions = random_tuned_model.predict(X_test)

# Score the predictions with mse and r2
mse = mean_squared_error(y_test, random_tuned_predictions)
r2 = r2_score(y_test, random_tuned_predictions)

print(f"Mean Squared Error (MSE): {mse}")
print(f"R-squared (R2 ): {r2}")


Mean Squared Error (MSE): 78.12047558024533
R-squared (R2 ): 0.5285124961526573


### Grid Search 

In [25]:
# Start grid search based on best parameters from random search

from sklearn.model_selection import GridSearchCV

params = {'bootstrap': [True],
         'max_depth': [8,10,15],
         'max_features': ['sqrt'],
         'min_samples_leaf': [3, 4, 6],
         'min_samples_split': [8,10,20],
         'n_estimators': [1500,2000,3000]}

rf_grid = GridSearchCV(estimator = RandomForestRegressor(), param_grid = params, cv = 3, n_jobs = -1, verbose = 2)
rf_grid.fit(X_train, y_train)


Fitting 3 folds for each of 81 candidates, totalling 243 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  33 tasks      | elapsed:  1.4min
[Parallel(n_jobs=-1)]: Done 154 tasks      | elapsed:  5.9min
[Parallel(n_jobs=-1)]: Done 243 out of 243 | elapsed:  9.3min finished


GridSearchCV(cv=3, estimator=RandomForestRegressor(), n_jobs=-1,
             param_grid={'bootstrap': [True], 'max_depth': [8, 10, 15],
                         'max_features': ['sqrt'],
                         'min_samples_leaf': [3, 4, 6],
                         'min_samples_split': [8, 10, 20],
                         'n_estimators': [1500, 2000, 3000]},
             verbose=2)

In [26]:
rf_grid.best_params_

{'bootstrap': True,
 'max_depth': 15,
 'max_features': 'sqrt',
 'min_samples_leaf': 3,
 'min_samples_split': 20,
 'n_estimators': 2000}

In [27]:
grid_tuned_model = rf_grid.best_estimator_

In [28]:
grid_tuned_predictions = grid_tuned_model.predict(X_test)

# Score the predictions with mse and r2
mse = mean_squared_error(y_test, grid_tuned_predictions)
r2 = r2_score(y_test, grid_tuned_predictions)

print(f"Mean Squared Error (MSE): {mse}")
print(f"R-squared (R2 ): {r2}")

Mean Squared Error (MSE): 78.33638994277476
R-squared (R2 ): 0.5272093688601304


### Final Machine Learning Model

In [29]:
# Note: at this point we're not getting much benefit from continued tuning
final_model=grid_tuned_model


In [30]:
# feature importance
sorted(zip(final_model.feature_importances_, feature_names), reverse=True)


[(0.4538646927645108, 'SVI_sum_of_indicators'),
 (0.2413342034655859, 'percent_uninsured'),
 (0.07022250706533824, 'clinics_per_capita'),
 (0.06362530271323628, 'health_dept_per_capita'),
 (0.04054222436357061, 'dept_clinic_title10_per_capita'),
 (0.037945702178360505, 'title10_clinics_per_capita'),
 (0.036245661483306164, 'title_10_clients_per_capita'),
 (0.02743163246927123, 'pp_clients_per_capita'),
 (0.024643850984248372, 'pp_per_capita'),
 (0.004144222512571778, 'hospitals_per_capita')]

## Pickle the Model

In [32]:
# Dump the model into a pkl document 
pickle.dump(final_model, open("model.pkl", "wb"))


### Check the Model 

In [33]:
model = pickle.load(open("model.pkl", "rb"))

In [34]:
print(model)

RandomForestRegressor(max_depth=15, max_features='sqrt', min_samples_leaf=3,
                      min_samples_split=20, n_estimators=2000)
